In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sohamjadiye","key":"2c8283fc5d29cc30347b9776e59e53bc"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/


In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 89% 23.0M/25.7M [00:00<00:00, 241MB/s]
100% 25.7M/25.7M [00:00<00:00, 249MB/s]


In [7]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense,Dropout,BatchNormalization
from sklearn.model_selection import train_test_split

In [8]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [9]:
df = pd.read_csv("IMDB Dataset.csv")

In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
sentences = df['review']
labels = df['sentiment'].map({'negative': 0, 'positive': 1})

In [12]:
labels

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sentences), np.array(labels), train_size=0.8, random_state=42)

In [14]:
print(f'X train Shape : {X_train.shape}')
print(f'y train Shape : {y_train.shape}')
print()
print(f'X test Shape : {X_test.shape}')
print(f'y test Shape : {y_test.shape}')

X train Shape : (40000,)
y train Shape : (40000,)

X test Shape : (10000,)
y test Shape : (10000,)


In [15]:
total_words = 1000
max_length = 120

In [16]:
tokenizer = Tokenizer(num_words=total_words,oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [17]:
len(tokenizer.word_index)

112281

In [18]:
train_seq = tokenizer.texts_to_sequences(X_train)
train_padd = pad_sequences(train_seq,maxlen=max_length,truncating='post')

In [19]:
test_seq = tokenizer.texts_to_sequences(X_test)
test_padd = pad_sequences(test_seq,maxlen=max_length,truncating='post')

In [20]:
max_seq_len = max([len(x) for x in train_seq])
max_seq_len

2493

In [21]:
index = 5
print(f'Text :\n{X_train[index]}\n\n')
print(f'Sequence :\n{train_seq[index]}\n\n')
print(f'Padded Sequence :\n{train_padd[index]}\n\n')

Text :
I don't know how this movie received so many positive reviews on this site! I'm a big vonnegut fan and am very familiar with the story this Showtime original film bastardized beyond belief, but even if I wasn't, the poor acting, VERY poor casting (Sean Astin as the brilliant, athletic, and all around individual, Harrison?? The guy's completely generic!) and sub-standard writing rendered this tripe barely watchable. Someone pointed out how cute that Maculay Culkin line was. If you read that and thought that was pure comic brilliance (sadly, it probably was the most INTENTIONALLY funny part of the movie), maybe you'll like this movie. But if you're a Vonnegut fan or not completely insane, don't see it. Please.


Sequence :
[11, 90, 119, 87, 12, 18, 1, 35, 108, 1, 826, 21, 12, 1, 145, 4, 194, 1, 324, 3, 236, 52, 1, 17, 2, 63, 12, 1, 210, 20, 1, 667, 1, 19, 58, 44, 11, 292, 2, 334, 113, 52, 334, 1, 1, 1, 15, 2, 512, 1, 3, 30, 184, 1, 1, 2, 1, 338, 1, 3, 1, 1, 487, 1, 12, 1, 1, 1, 29

In [22]:
index = 5
print(f'Text :\n{X_test[index]}\n\n')
print(f'Sequence :\n{test_seq[index]}\n\n')
print(f'Padded Sequence :\n{test_padd[index]}\n\n')

Text :
I've watched this movie on a fairly regular basis for most of my life, and it never gets old. For all the snide remarks and insults (mostly from David Spade), "Tommy Boy" has a giant heart. And that's what keeps this movie funny after all these years.<br /><br />Tommy Callahan (Chris Farley) is the son of Big Tom Callahan (Brian Dennehy), master car parts salesman, and has ridden on that all his life. But after his died dies on his wedding day, Tommy learns that the company is in debt, and about to be bought by Ray Zalinsky (Dan Akroyd), the owner of a huge car parts company. So in order to save the company, Tommy has to go on the road to sell the company's new brake pads. Along for the ride, though not by choice, is Richard Hayden (David Spade) a former classmate of Tommy's who was Big Tom's right-hand man.<br /><br />The movie rides on the chemistry between the two SNL stars (and real-life best friends) Chris Farley and David Spade. The duo has enough comic energy going betwee

In [23]:
embedding_dim = 16
model = Sequential([
        Embedding(total_words,embedding_dim,input_length=max_length),
        Bidirectional(LSTM(64,return_sequences=True)),
        Bidirectional(LSTM(32)),

        Dense(6,activation='relu'),
        Dense(1,activation='sigmoid')
])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           16000     
                                                                 
 bidirectional (Bidirection  (None, 120, 128)          41472     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 99085 (387.05 KB)
Trainable params: 99085 

In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau ,EarlyStopping
callbacks = [
            EarlyStopping(patience = 10, verbose=1),
            ReduceLROnPlateau(factor = 0.5, patience = 3, min_lr = 0.000001, verbose=1),
            ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True,
            verbose=1)

    ]

In [26]:
model.compile(optimizer=Adam(0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
history = model.fit(train_padd,y_train,epochs=20,validation_data=(test_padd,y_test),callbacks=[callbacks])

Epoch 1/20
1250/1250 [==============================] - ETA: 0s - loss: 0.5449 - accuracy: 0.7125
Epoch 1: val_loss improved from inf to 0.42374, saving model to model.h5
1250/1250 [==============================] - 57s 36ms/step - loss: 0.5449 - accuracy: 0.7125 - val_loss: 0.4237 - val_accuracy: 0.8046 - lr: 0.0010
Epoch 2/20
   3/1250 [..............................] - ETA: 37s - loss: 0.4581 - accuracy: 0.7917

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - ETA: 0s - loss: 0.4283 - accuracy: 0.8019
Epoch 2: val_loss improved from 0.42374 to 0.41478, saving model to model.h5
1250/1250 [==============================] - 28s 23ms/step - loss: 0.4283 - accuracy: 0.8019 - val_loss: 0.4148 - val_accuracy: 0.8108 - lr: 0.0010
Epoch 3/20
1248/1250 [============================>.] - ETA: 0s - loss: 0.5273 - accuracy: 0.7362
Epoch 3: val_loss did not improve from 0.41478
1250/1250 [==============================] - 26s 21ms/step - loss: 0.5273 - accuracy: 0.7362 - val_loss: 0.4459 - val_accuracy: 0.7961 - lr: 0.0010
Epoch 4/20
1250/1250 [==============================] - ETA: 0s - loss: 0.4248 - accuracy: 0.8056
Epoch 4: val_loss improved from 0.41478 to 0.40900, saving model to model.h5
1250/1250 [==============================] - 26s 21ms/step - loss: 0.4248 - accuracy: 0.8056 - val_loss: 0.4090 - val_accuracy: 0.8121 - lr: 0.0010
Epoch 5/20
1248/1250 [============================>.] - ETA: 0s - loss: 

In [28]:
model.evaluate(test_padd,y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.3992 - accuracy: 0.8267


[0.3992016017436981, 0.82669997215271]